In [3]:
pip install tensorflow keras scikit-learn pandas pillow

In [ ]:
# ------------------------------
# Import libraries
# ------------------------------
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from PIL import Image
import cv2

# ------------------------------
# Parameters and paths
# ------------------------------
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNELS = 1
BATCH_SIZE = 32
EPOCHS = 5

BASE_PATH = '/content/drive/MyDrive/obj/salt'
TRAIN_PATH = os.path.join(BASE_PATH, 'train')
TEST_PATH = os.path.join(BASE_PATH, 'test')
TRAIN_CSV = os.path.join(BASE_PATH, 'train.csv')
DEPTHS_CSV = os.path.join(BASE_PATH, 'depths.csv')
SAMPLE_SUBMISSION_CSV = os.path.join(BASE_PATH, 'sample_submission.csv')

# ------------------------------
# RLE functions
# ------------------------------
def rle_decode(mask_rle, shape=(101,101)):
    if mask_rle == '' or pd.isna(mask_rle):
        return np.zeros(shape)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def rle_encode(img):
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)

# ------------------------------
# Dice metric
# ------------------------------
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

# ------------------------------
# Data loading
# ------------------------------
def load_and_preprocess_data():
    train_df = pd.read_csv(TRAIN_CSV)
    depths_df = pd.read_csv(DEPTHS_CSV)
    train_df = train_df.merge(depths_df, on='id')

    images, masks = [], []
    for _, row in train_df.iterrows():
        img_path = os.path.join(TRAIN_PATH, 'images', row['id'] + '.png')
        img = load_img(img_path, color_mode='grayscale', target_size=(IMG_HEIGHT, IMG_WIDTH))
        img = img_to_array(img) / 255.0
        images.append(img)

        mask = rle_decode(row['rle_mask'])
        mask = Image.fromarray(mask).resize((IMG_HEIGHT, IMG_WIDTH))
        mask = img_to_array(mask) / 255.0
        masks.append(mask)

    return np.array(images), np.array(masks)

def load_test_data():
    test_df = pd.read_csv(SAMPLE_SUBMISSION_CSV)
    test_images, test_ids = [], []
    for _, row in test_df.iterrows():
        img_path = os.path.join(TEST_PATH, 'images', row['id'] + '.png')
        img = load_img(img_path, color_mode='grayscale', target_size=(IMG_HEIGHT, IMG_WIDTH))
        img = img_to_array(img) / 255.0
        test_images.append(img)
        test_ids.append(row['id'])
    return np.array(test_images), test_ids

# ------------------------------
# Data generator
# ------------------------------
def data_generator(images, masks, batch_size=16, augment=True):
    num_samples = len(images)
    while True:
        indices = np.random.permutation(num_samples)
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_images = images[batch_indices]
            batch_masks = masks[batch_indices]

            if augment:
                for j in range(len(batch_images)):
                    if np.random.rand() > 0.5:
                        batch_images[j] = np.fliplr(batch_images[j])
                        batch_masks[j] = np.fliplr(batch_masks[j])
                    if np.random.rand() > 0.5:
                        factor = 0.5 + np.random.rand()
                        batch_images[j] = np.clip(batch_images[j] * factor, 0, 1)
            yield batch_images, batch_masks

# ------------------------------
# U-Net model
# ------------------------------
def unet_model():
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

    # Contracting path
    c1 = Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    c1 = Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal')(c1)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    p1 = MaxPooling2D((2,2))(c1)
    p1 = Dropout(0.1)(p1)

    c2 = Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    c2 = Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal')(c2)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    p2 = MaxPooling2D((2,2))(c2)
    p2 = Dropout(0.1)(p2)

    c3 = Conv2D(256, (3,3), padding='same', kernel_initializer='he_normal')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    c3 = Conv2D(256, (3,3), padding='same', kernel_initializer='he_normal')(c3)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    p3 = MaxPooling2D((2,2))(c3)
    p3 = Dropout(0.2)(p3)

    c4 = Conv2D(512, (3,3), padding='same', kernel_initializer='he_normal')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    c4 = Conv2D(512, (3,3), padding='same', kernel_initializer='he_normal')(c4)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    p4 = MaxPooling2D((2,2))(c4)
    p4 = Dropout(0.2)(p4)

    # Bottleneck
    c5 = Conv2D(1024, (3,3), padding='same', kernel_initializer='he_normal')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)
    c5 = Conv2D(1024, (3,3), padding='same', kernel_initializer='he_normal')(c5)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)
    c5 = Dropout(0.3)(c5)

    # Expansive path
    u6 = UpSampling2D((2,2))(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3,3), padding='same', kernel_initializer='he_normal')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)

    u7 = UpSampling2D((2,2))(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3,3), padding='same', kernel_initializer='he_normal')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('relu')(c7)

    u8 = UpSampling2D((2,2))(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('relu')(c8)

    u9 = UpSampling2D((2,2))(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Activation('relu')(c9)

    outputs = Conv2D(1, (1,1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=Adam(1e-4), loss=dice_loss, metrics=['accuracy', dice_coef])
    return model

# ------------------------------
# Training
# ------------------------------
X, y = load_and_preprocess_data()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = unet_model()
callbacks = [
    EarlyStopping(patience=15, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=10, min_lr=1e-7),
    ModelCheckpoint('salt_model_best.h5', save_best_only=True)
]

history = model.fit(
    data_generator(X_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train)//BATCH_SIZE,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    callbacks=callbacks
)

# ------------------------------
# Segmentation (Prediction)
# ------------------------------
model = load_model("salt_model_best.h5", custom_objects={"dice_coef": dice_coef, "dice_loss": dice_loss})
test_images, test_ids = load_test_data()
preds = model.predict(test_images, batch_size=BATCH_SIZE, verbose=1)

# Threshold to binary masks
pred_masks = (preds > 0.5).astype(np.uint8)

# ------------------------------
# Submission file
# ------------------------------
submission = pd.read_csv(SAMPLE_SUBMISSION_CSV)
for pred, test_id in zip(pred_masks, test_ids):
    pred_resized = cv2.resize(np.squeeze(pred), (101, 101))
    rle = rle_encode(pred_resized)
    submission.loc[submission['id'] == test_id, 'rle_mask'] = rle

submission.to_csv("submission.csv", index=False)
print("Submission saved: submission.csv")

# ------------------------------
# Visualization
# ------------------------------
fig, axes = plt.subplots(3, 3, figsize=(12,12))
for i in range(3):
    axes[i,0].imshow(np.squeeze(test_images[i]), cmap="gray")
    axes[i,0].set_title("Original")
    axes[i,0].axis("off")

    axes[i,1].imshow(np.squeeze(preds[i]), cmap="jet")
    axes[i,1].set_title("Predicted Prob")
    axes[i,1].axis("off")

    axes[i,2].imshow(np.squeeze(pred_masks[i]), cmap="gray")
    axes[i,2].set_title("Segmentation")
    axes[i,2].axis("off")

plt.tight_layout()
plt.savefig("segmentation_results.png")
plt.show()


Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.5312 - dice_coef: 0.0033 - loss: 0.9967

100/100 ━━━━━━━━━━━━━━━━━━━━ 65s 441ms/step - accuracy: 0.5323 - dice_coef: 0.0033 - loss: 0.9967 - val_accuracy: 5.2727e-04 - val_dice_coef: 0.0020 - val_loss: 0.9980 - learning_rate: 1.0000e-04
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step - accuracy: 0.7207 - dice_coef: 0.0051 - loss: 0.9949

100/100 ━━━━━━━━━━━━━━━━━━━━ 48s 482ms/step - accuracy: 0.7207 - dice_coef: 0.0051 - loss: 0.9949 - val_accuracy: 0.0611 - val_dice_coef: 0.0021 - val_loss: 0.9979 - learning_rate: 1.0000e-04
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.7271 - dice_coef: 0.0056 - loss: 0.9944

100/100 ━━━━━━━━━━━━━━━━━━━━ 47s 470ms/step - accuracy: 0.7272 - dice_coef: 0.0056 - loss: 0.9944 - val_accuracy: 0.4020 - val_dice_coef: 0.0031 - val_loss: 0.9969 - learning_rate: 1.0000e-04
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step - accuracy: 0.7429 - dice_coef: 0.0060 - loss: 0.9940

100/100 ━━━━━━━━━━━━━━━━━━━━ 49s 491ms/step - accuracy: 0.7429 - dice_coef: 0.0060 - loss: 0.9940 - val_accuracy: 0.6204 - val_dice_coef: 0.0047 - val_loss: 0.9953 - learning_rate: 1.0000e-04
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step - accuracy: 0.7451 - dice_coef: 0.0064 - loss: 0.9936

100/100 ━━━━━━━━━━━━━━━━━━━━ 47s 472ms/step - accuracy: 0.7451 - dice_coef: 0.0064 - loss: 0.9936 - val_accuracy: 0.6975 - val_dice_coef: 0.0057 - val_loss: 0.9943 - learning_rate: 1.0000e-04
